In [1]:
import re
# nltk?
from english_words import get_english_words_set

wl = 5  # word length
# getting all wl-len words for wordle
words = [word for word in get_english_words_set(['web2'], lower=True) if len(word) == wl]
print("found", len(words), "words")
words = ' '.join(words)

found 9979 words


In [8]:
def match_words(to_find=".....", bad_c="", good_c="", guesses=[], verbose=False):
    """ gets a list of all wl-letter words given criteria 
        to_find => the word matches these letters in this order with '.' for unknown
        bad_c => the word doesnt contain these characters
        good_c => the word does contain these characters"""
    # get all words that match this sequence
    found_words = re.findall(to_find, words)
#     if verbose: 
#         print("Found words:", found_words)

#     print("hound" in found_words)  # TODO figure why this isnt working

    # character subset
    bad_c += ' '  # addition for space delim in regex
    res = []
    for word in found_words:
        can_add = True
        for c in bad_c:  # subtract words with bad characters
            if c in word:
                can_add = False
        if can_add:
            for c in good_c:  # add words with good characters
                if c not in word:
                    can_add = False
        if can_add and word in guesses:
            can_add = False  # TODO: elim words with - feedback matching letters
        if can_add:
            res.append(word)
    if verbose:
        print("\nResulting words:")
        print(res)
    return res

potential_words = match_words(".....", "arise", "", [], True)  # easiest


Resulting words:
['yucky', 'toddy', 'podgy', 'humph', 'boggy', 'wuzzy', 'fulth', 'clock', 'bulky', 'tucum', 'cooly', 'onymy', 'bungy', 'choco', 'conoy', 'lymph', 'puggy', 'poddy', 'pondo', 'woozy', 'boonk', 'yummy', 'bundy', 'floyd', 'fouth', 'goofy', 'duchy', 'cloth', 'mooch', 'pouty', 'bombo', 'mogul', 'glynn', 'plumb', 'bonbo', 'thoft', 'octyl', 'hoddy', 'poppy', 'thuoc', 'dunch', 'fully', 'gully', 'blowy', 'notum', 'bolly', 'fluty', 'boldu', 'lotto', 'yolky', 'fungo', 'hobby', 'bothy', 'uncoy', 'jumbo', 'totum', 'holly', 'jocko', 'mothy', 'jotty', 'woody', 'jocum', 'glout', 'coyly', 'cloot', 'lludd', 'cumbu', 'mooth', 'punct', 'poult', 'bluff', 'oghuz', 'woofy', 'honzo', 'unbud', 'muzzy', 'goudy', 'bumbo', 'notch', 'cocco', 'hoppy', 'lolly', 'montu', 'uncut', 'cocky', 'howff', 'bubby', 'ocuby', 'golly', 'compo', 'clomb', 'kongu', 'gubbo', 'oolly', 'outly', 'lunch', 'pocky', 'louty', 'comby', 'outdo', 'xylyl', 'touch', 'potch', 'fuffy', 'ghoul', 'bulby', 'hokum', 'buddh', 'mummy', 

In [16]:
def best_play(good_words, verbose=False):
    """ returns the best play for wordle based on character frequency 
    Note: this is not necessarily optimal since word difficulty needs to be taken into account
     (i.e. 'later' is better then 'artie' because nobody gona put artie in wordle XD)"""
    freq = {}  # get frequency map of all characters
    freq2 = [{} for i in range(wl)]   # get frequency map of character frequency in each position
    for word in good_words:  # TODO: weight freq by how common the word is
        for c in set(word): # save raw freq
            freq[c] = freq.get(c, 0) + 1
        for i, c in enumerate(word):  # save freq for specific places
            freq2[i][c] = freq2[i].get(c, 0) + 1
    if verbose:
        print("*Frequency Map")  # show in sorted order
        print({k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)})
        for i in range(len(freq2)):
            print("Frequency2 Map", i)  # show in sorted order
            print({k: v for k, v in sorted(freq2[i].items(), key=lambda item: item[1], reverse=True)})
    
    score = {}  # get score of each word
    for word in good_words:
        sc = 0
        for c in set(word):  # add freq of letter occurring in any word
            sc += freq[c]  # aka chance of -
        for i, c in enumerate(word):  # add freq of letter occurring in that spot in any word (half weighted)
            sc += freq2[i].get(c, 0)/1.5 # aka chance of * 
        score[word] = sc  # TODO: account for previously guessed words
        
    # tunable modifiers to words
    for word in score.keys():
        if word[-1] == "s":  # words ending in s are less frequently used in wordle
            score[word] = score[word] * 0.6
        elif word[-1] == "y":  # words ending in y are slightly less frequently used in wordle
            score[word] = score[word] * 0.8
            
        unq = {}  # words with < 3 unique characters are difficult and are less frequently used in wordle
        for c in word:
            unq[c] = 1
        if len(unq) < 3:
            score[word] = score[word] * 0.7
    # TODO: word freq - penalize words that are super frequent or super esoteric

    # sort possible words based on score
    res = sorted(good_words, key=lambda word: score[word], reverse=True)
    if verbose:
        print("\nWords sorted by best option:")
        print(res)
    return res

play = best_play(potential_words, True)[0]  # words.split(" ")

*Frequency Map
{'o': 82, 'u': 59, 'y': 57, 'l': 38, 'c': 33, 'h': 30, 't': 29, 'b': 27, 'm': 26, 'n': 20, 'd': 17, 'g': 16, 'p': 15, 'f': 12, 'k': 11, 'w': 7, 'z': 5, 'j': 4, 'x': 1}
Frequency2 Map 0
{'b': 17, 'c': 13, 'p': 11, 'h': 10, 'l': 8, 'g': 8, 'f': 7, 'o': 7, 'm': 7, 't': 6, 'w': 4, 'j': 4, 'y': 3, 'u': 3, 'd': 2, 'n': 2, 'k': 1, 'x': 1}
Frequency2 Map 1
{'o': 58, 'u': 31, 'l': 12, 'n': 4, 'h': 4, 'y': 2, 'c': 2, 'g': 1}
Frequency2 Map 2
{'o': 18, 'l': 13, 'u': 13, 'c': 12, 'n': 12, 'm': 11, 't': 11, 'd': 5, 'b': 4, 'g': 3, 'y': 3, 'z': 2, 'p': 2, 'w': 2, 'h': 1, 'f': 1, 'k': 1}
Frequency2 Map 3
{'d': 12, 'u': 12, 'l': 12, 'b': 12, 't': 10, 'c': 10, 'k': 7, 'g': 6, 'f': 6, 'p': 5, 'm': 5, 'z': 4, 'o': 4, 'y': 3, 'h': 3, 'n': 2, 'w': 1}
Frequency2 Map 4
{'y': 52, 'o': 16, 'h': 13, 'm': 6, 't': 6, 'u': 5, 'l': 4, 'd': 3, 'k': 2, 'b': 2, 'f': 2, 'n': 1, 'c': 1, 'z': 1}

Words sorted by best option:
['locum', 'touch', 'boldu', 'poult', 'louty', 'fouth', 'montu', 'humbo', 'mogul', 

In [23]:
def play_wordle(target, guess):
    """ executes 1 round of wordle and returns feedback """
    assert len(target) == wl and len(guess) == wl
    res = ""
    for i in range(len(target)):
        if target[i] == guess[i]:
            res += '*'
        elif guess[i] in target:
            res += '-'
        else:
            res += 'x'
    return res

def wordle(target, max_guess=6):
    """ executes max_guess rounds of wordle and tells you if you won :) 
    (returns all your guesses & feedback) """
    i = 0
    results = []
    while i <= max_guess:
        inword = str(input("Your Guess: ")).lower().strip()
        feedback = play_wordle(target, inword)
        results.append((inword, feedback))
        print("Feedback: ", feedback)
        print()
        if feedback == '*****':
            print("Congrats! You win :)\n")
            return results
        i += 1
    print("Sorry thats not right...\nThe correct word was:", target)
    return results
test = wordle("gummy")

Your Guess: raise
Feedback:  xxxxx

Your Guess: clout
Feedback:  xxx-x

Your Guess: thumb
Feedback:  xx-*x

Your Guess: tummy
Feedback:  x****

Your Guess: gummy
Feedback:  *****

Congrats! You win :)



[('raise', 'xxxxx'),
 ('clout', 'xxx-x'),
 ('thumb', 'xx-*x'),
 ('tummy', 'x****'),
 ('gummy', '*****')]

In [68]:
from numpy.random import choice

pool_size = 20  # max deviation from best choice
rand_weight = 1.5  # int [1,inf] exponentially scaling how likely sorted choices are to each other
def bot_wordle(target, num_guesses=6, verbose=False):
    """ shows wordle guesses & results for a freq-optimal bot
    Note: its not very good for reasons discussed earlier """
    counter = 1
    potential_words = words.split(' ')
    guesses = []
    # memory variables between words (can be sets but im lazy)
    good_c = ""
    bad_c = ""
    while counter <= num_guesses:
        assert len(potential_words) > 0, f"No choices found in {potential_words=}"
        
        # random choice of good words is more interesting than always taking the best word
        # tldr masks some of the pitfalls of the strategy in variability
        best_words = best_play(potential_words)
        if len(best_words) > pool_size:  # constrain random choice
            best_words = best_words[:pool_size]
        inword = choice(best_words, 1, [rand_weight**i for i in reversed(range(len(best_words)))])[0]
        
        guesses.append(inword)
        print("Guessed:", inword)
        
        feedback = play_wordle(target, inword)
        print("Feedback:", feedback)
        
        if feedback == '*****':
            print("Congrats! You win :)")
            return
        else:  # find next best option
            to_find = ""
            for i, c in enumerate(feedback):  # update params based on feedback
                if c == "*":
                    to_find += inword[i]
                else:
                    to_find += '.'
                    if c == "-":
                        good_c += inword[i]
                    else:
                        bad_c += inword[i]
            # update words for next round
            if verbose: print("Update:", ', '.join([to_find, bad_c, good_c]))
            potential_words = match_words(to_find, bad_c, good_c, guesses)
        print()
        counter += 1
    print("Sorry thats not right...\nThe correct word was:", target)
    return

bot_wordle("found", 6, True)

# TODO: make it so bot looks at previous guesses and knows - letters can be in that position*

Guessed: ariel
Feedback: xxxxx
Update: ....., ariel, 
False

Guessed: booty
Feedback: x*-xx
Update: .o..., arielbty, o
False

Guessed: donum
Feedback: -*--x
Update: .o..., arielbtym, odnu
False

Guessed: fondu
Feedback: **---
Update: fo..., arielbtym, odnundu
False

Guessed: found
Feedback: *****
Congrats! You win :)
